In [1]:
#import all dependancies 
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re

In [2]:
headlines = []
snippets = []
dates = []

for n in range(3000):
    
    # Reuters news headline for tech page url
    reuters_headline_url = "https://www.reuters.com/news/archive/technologyNews?view=page&page={}&pageSize=10".format(n)
    #print(reuters_headline_url)
    
    # Retrieve page with the requests module
    response = requests.get(reuters_headline_url)
    
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(response.text, 'html.parser')
    
    # results are returned as an iterable list
    results = soup.find_all('article', class_="story")
    
    # Loop through returned results
    counter = 1
    for result in results:

        if (counter<11):
            
            # Identify and return title of listing
            try:
                headline = result.find('h3', class_="story-title").text
                headlines.append(headline)
            except:
                headlines.append('missing')

            # Identify and return paragraph of headline
            try:
                para = result.p.text
                snippets.append(para)
            except:
                snippets.append('missing')

            # Identify and return the date of the headline
            try:
                date = result.find('time',class_ = "article-time").span.text
                dates.append(date)
                #print(date)
            except:
                dates.append('missing')

            counter+=1

In [3]:
#write the lists to a dataframe
reuters_df = pd.DataFrame({'Date': dates, 'Headline' : headlines, 'article_snippet' : snippets})

reuters_df.head()

,Date,Headline,article_snippet
0,11:32am EDT,\n\t\t\t\t\t\t\t\tTwitter shares rise on recor...,Twitter Inc on Thursday reported its highest-...
1,10:51am EDT,\n\t\t\t\t\t\t\t\tFacebook allows users to bro...,Facebook Inc said on Thursday it was rolling ...
2,10:41am EDT,\n\t\t\t\t\t\t\t\tAT&T loses monthly subscribe...,AT&T Inc said on Thursday it lost monthly phon...
3,10:13am EDT,\n\t\t\t\t\t\t\t\tAmazon to open 10 new India ...,Amazon.com Inc's India unit on Thursday said ...
4,9:53am EDT,\n\t\t\t\t\t\t\t\tOrange launches digital bank...,French telecoms provider Orange on Thursday la...


In [4]:
reuters_df = reuters_df.apply(lambda x: x.str.strip())
reuters_df.head()

,Date,Headline,article_snippet
0,11:32am EDT,Twitter shares rise on record yearly growth in...,Twitter Inc on Thursday reported its highest-...
1,10:51am EDT,Facebook allows users to broadcast live large ...,Facebook Inc said on Thursday it was rolling ...
2,10:41am EDT,AT&T loses monthly subscribers as media unit h...,AT&T Inc said on Thursday it lost monthly phon...
3,10:13am EDT,Amazon to open 10 new India warehouses; offers...,Amazon.com Inc's India unit on Thursday said ...
4,9:53am EDT,Orange launches digital banking in Ivory Coast...,French telecoms provider Orange on Thursday la...


In [61]:
 #write to a file
#reuters_df.to_csv("../data/reuter_tech_data.csv")

In [12]:
reuter_df = reuters_df.drop_duplicates(subset=['Headline', 'article_snippet'], keep='first')

In [17]:
reuter_df.at[0:23,'Date'] = "Jul 23 2020"

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2894: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[index, col] = value


In [19]:
reuter_df.head()

,Date,Headline,article_snippet
0,Jul 23 2020,Twitter shares rise on record yearly growth in...,Twitter Inc on Thursday reported its highest-...
1,Jul 23 2020,Facebook allows users to broadcast live large ...,Facebook Inc said on Thursday it was rolling ...
2,Jul 23 2020,AT&T loses monthly subscribers as media unit h...,AT&T Inc said on Thursday it lost monthly phon...
3,Jul 23 2020,Amazon to open 10 new India warehouses; offers...,Amazon.com Inc's India unit on Thursday said ...
4,Jul 23 2020,Orange launches digital banking in Ivory Coast...,French telecoms provider Orange on Thursday la...


In [20]:
reuter_df['Date'] = reuter_df['Date'].astype('datetime64[ns]') 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
reuter_df.head()

,Date,Headline,article_snippet
0,2020-07-23,Twitter shares rise on record yearly growth in...,Twitter Inc on Thursday reported its highest-...
1,2020-07-23,Facebook allows users to broadcast live large ...,Facebook Inc said on Thursday it was rolling ...
2,2020-07-23,AT&T loses monthly subscribers as media unit h...,AT&T Inc said on Thursday it lost monthly phon...
3,2020-07-23,Amazon to open 10 new India warehouses; offers...,Amazon.com Inc's India unit on Thursday said ...
4,2020-07-23,Orange launches digital banking in Ivory Coast...,French telecoms provider Orange on Thursday la...


In [22]:
reuter1 = reuter_df.groupby(['Date'])['Headline'].apply(','.join).reset_index()

In [23]:
reuter2 = reuter_df.groupby(['Date'])['article_snippet'].apply(','.join).reset_index()

In [36]:
reuter_data = pd.merge(reuter1, reuter2, on="Date", how="outer")

In [37]:
reuter_data.tail()

,Date,Headline,article_snippet
2011,2020-07-19,Symptom tracker app reveals six distinct types...,(This July 17 story corrects paragraph seven t...
2012,2020-07-20,IBM beats estimates; sees gains as customers a...,International Business Machines Corp beat est...
2013,2020-07-21,Twitter will suspend accounts tweeting about c...,Twitter Inc said on Tuesday it would permanen...
2014,2020-07-22,U.S. appeals court won't unseal papers in Face...,A federal appeals court on Wednesday declined ...
2015,2020-07-23,Twitter shares rise on record yearly growth in...,Twitter Inc on Thursday reported its highest-...


In [9]:
import time
headlines = []
snippets = []
dates = []

for n in range(0,3000,15):
    
    # Tech News World headline for tech page url
    technewsworld_headline_url = "https://www.technewsworld.com/perl/archives/tnw/?init={}".format(n)
    #print(technewsworld_headline_url)

    # Retrieve page with the requests module
    response = requests.get(technewsworld_headline_url)

    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(response.text, 'html.parser')

    # results are returned as an iterable list
    results = soup.find_all('div', class_="story-list xlarge shadow-big")

    # Loop through returned results
    for result in results:

        # Identify and return title of listing
        try:
            headline = result.find('div', class_="title").text
            headlines.append(headline)
            #print(headline)
        except:
            headlines.append('missing')

        # Identify and return paragraph of headline
        try:
            para = result.find('div', class_="teaser").text
            snippets.append(para)
            #print(para)
        except:
            snippets.append('missing')

        # Identify and return the date of the headline
        try:
            date = result.find('div',class_ = "date").text
            dates.append(date)
            #print(date)
        except:
            dates.append('missing')
    time.sleep(0.1)

In [10]:
#write the lists to a dataframe
technewsworld_df = pd.DataFrame({'Date': dates, 'Headline' : headlines, 'article_snippet' : snippets})

technewsworld_df.tail()

,Date,Headline,article_snippet
1890,"Wednesday - July 23, 2014","Amazon Fire Phone Wins Respect, if Not Raves",Early reviews for Amazon's first foray into th...
1891,"Tuesday - July 22, 2014",Nvidia's Shield Tablet Earns Praise for Hardwa...,Nvidia is expanding its Shield line of gaming ...
1892,"Tuesday - July 22, 2014",GoTenna Makes Wireless 'Magic',Startup GoTenna has launched its eponymously n...
1893,"Friday - April 11, 2014",Consumers Can't Stanch Heartbleeding,Consumers can do little to protect themselves ...
1894,"Thursday - February 27, 2014",Boeing Unveils Suicidally Secure Smartphone,Boeing this week filed an application with the...


In [29]:
technewsworld_df['Date'] = technewsworld_df['Date'].astype('datetime64[ns]') 

In [30]:
technewsworld_df.head()

,Date,Headline,article_snippet
0,2020-07-22,US Cloud Gaming Subscriptions Target $3B,Consumer interest in cloud gaming could boost ...
1,2020-07-21,Windows 10X to Eliminate Onboard 32-Bit Apps: ...,Windows 10X began as a streamlined operating s...
2,2020-07-21,"ANALYSISThe Past, Present and Future of Attrib...",Encryption is an obscure but critical part of ...
3,2020-07-20,OPINIONWhy Microsoft Teams Will Be the De Fact...,I've been covering video conferencing since be...
4,2020-07-17,3 Apps to Help Golfers Get Into the Swing of T...,Sports performance technology continues to adv...


In [31]:
technewsworld1 = technewsworld_df.groupby(['Date'])['Headline'].apply(','.join).reset_index()

In [32]:
technewsworld2 = technewsworld_df.groupby(['Date'])['article_snippet'].apply(','.join).reset_index()

In [38]:
technewsworld_data = pd.merge(technewsworld1, technewsworld2, on="Date", how="outer")

In [40]:
technewsworld_data.tail()

,Date,Headline,article_snippet
1443,2020-07-15,Samsung Reveals 6G Vision,Although many people haven't wrapped their min...
1444,2020-07-17,3 Apps to Help Golfers Get Into the Swing of T...,Sports performance technology continues to adv...
1445,2020-07-20,OPINIONWhy Microsoft Teams Will Be the De Fact...,I've been covering video conferencing since be...
1446,2020-07-21,Windows 10X to Eliminate Onboard 32-Bit Apps: ...,Windows 10X began as a streamlined operating s...
1447,2020-07-22,US Cloud Gaming Subscriptions Target $3B,Consumer interest in cloud gaming could boost ...


In [42]:
news_data = pd.merge(reuter_data, technewsworld_data, on="Date", how='outer')

In [43]:
news_data.columns = ['Date','reuter_headline', 'reuter_article_summary','technewsworld_headline','technewsworld_article_summary']

In [44]:
news_data = news_data.sort_values(by='Date')

In [45]:
news_data.head()

,Date,reuter_headline,reuter_article_summary,technewsworld_headline,technewsworld_article_summary
2016,2014-02-27,NaN,NaN,Boeing Unveils Suicidally Secure Smartphone,Boeing this week filed an application with the...
2017,2014-04-11,NaN,NaN,Consumers Can't Stanch Heartbleeding,Consumers can do little to protect themselves ...
2018,2014-07-22,NaN,NaN,Nvidia's Shield Tablet Earns Praise for Hardwa...,Nvidia is expanding its Shield line of gaming ...
2019,2014-07-23,NaN,NaN,"Amazon Fire Phone Wins Respect, if Not Raves",Early reviews for Amazon's first foray into th...
2020,2014-07-28,NaN,NaN,OPINIONThe Skully Augmented Motorcycle Helmet ...,"Skully had its coming-out party last week, and..."


In [47]:
import datetime as dt
news_data = news_data.loc[(news_data.Date > dt.datetime.strptime('2014-09-17 00:00:00', '%Y-%m-%d %H:%M:%S'))]

In [48]:
news_data = news_data.astype(str)

In [49]:
news_data['all_news'] = news_data[['reuter_headline', 'reuter_article_summary','technewsworld_headline', 'technewsworld_article_summary']].apply(','.join, axis=1)

In [50]:
news_data = news_data.drop(columns =['reuter_headline', 'reuter_article_summary', 'technewsworld_headline', 'technewsworld_article_summary']) 

In [51]:
news_data['all_news'] = news_data['all_news'].str.lower()

In [52]:
news_data.head()

,Date,all_news
2051,2014-09-18,"nan,nan,amazon gets all fired up,harmony takes..."
2052,2014-09-19,"nan,nan,android l will keep your secrets safer..."
2053,2014-09-20,"nan,nan,gadget dreams and nightmaresgadget ogl..."
2054,2014-09-22,"nan,nan,sony's playstation tv could be a busy ..."
2055,2014-09-23,"nan,nan,pants on fire, schmidt tells climate c..."


In [53]:
news_data.to_csv("../data/reuter_technewsworld_data_merged.csv",index=False)